# Read in packages, define functions

Reads in all packages you may need

In [1]:
import matplotlib.pyplot as plt
try:
    import IPython
    shell = IPython.get_ipython()
    shell.enable_matplotlib(gui='qt')
except:
    pass


from MJOLNIR import _tools
from MJOLNIR.Data import DataSet
from lmfit import Model, Parameter, report_fit
from MJOLNIR.Data import Mask
from MJOLNIR._tools import fileListGenerator

import numpy as np
from os import path
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd
import copy

In [2]:
def generateBG(foregroundDataSet,backgroundDataSet, foregroundMask = None,dQ=0.02,dE=0.1,plotPowderAverage=False,verbose=True):
    """Generate a background dataset using the backgroundDataSet to perform powder averaged background interpolation
    in the shape of the foregroundDataSet.
    
    Args:
    
        - foregroundDataSet (DataSet): Foreground data set to be replicated by the bg
        
        - backgroundDataSet (DataSet): Masked data set to be used for background masking out intensities. 
        
    Kwargs: 
    
        - foregroundMask (Mask): Mask to apply to powder-averaged background dataset. If none use mask from foregroundDataSet (default None)
    
        - dQ (float): Bin size along Qlength (default 0.02 1/AA)
        
        - dE (float): Bin size in energy (default 0.1 meV)
        
        - plotPowderAverage (bool): Flag to control plotting of powder average, if true ax is also returned (default False)
        
        - verbose (bool): If true, print the averaging process to the command line (default True)
        
    
    """
    # Reset the mask of ds to be nothing
    bgmask = copy.deepcopy(backgroundDataSet.mask)
    backgroundDataSet.mask = [np.zeros_like(df.I,dtype=bool) for df in backgroundDataSet]
    
    # Find the maximal positions before the mask is applied
    qLength = np.array([np.max(np.linalg.norm([df.qx,df.qy],axis=0)) for df in backgroundDataSet])
    E = [[np.min(e),np.max(e)] for e in ds.energy.data]
    QMax = np.max(qLength) # QMin is always set to 0
    EMin = np.nanmin(E)
    EMax = np.nanmax(E)
    if verbose:
        print('Bin limites are\n','Q:',0.0,QMax,'\nE:',EMin,EMax)
        print('Copying bg dataset')
    
    backgroundDataSet.mask = bgmask
    
    # Copy the foreground data to bg data set
    newBackgroundDataSet = copy.deepcopy(foregroundDataSet)

    
    if verbose:
        print('Performing powder average')
    # Prepare for powder average of background data
    I = backgroundDataSet.I.extractData()
    Monitor = backgroundDataSet.Monitor.extractData()
    Norm = backgroundDataSet.Norm.extractData()
    
    # Position in the powder average (QLength, Energy)
    positions2D = np.array([np.linalg.norm([backgroundDataSet.qx.extractData(),
                                            backgroundDataSet.qy.extractData()],axis=0),
                            backgroundDataSet.energy.extractData()])

    # Generate the bins with suitable extensions
    QBins = np.arange(0,QMax+dQ*1.1,dQ)
    EnergyBins = np.arange(EMin-dE*1.1,EMax+dE*1.1,dE)

    
    # Perform 2D histogram
    normcounts,*powderBins = np.histogram2d(*positions2D,bins=np.array([QBins,EnergyBins],dtype=object),weights=np.ones((positions2D.shape[1])).flatten())
    intensity = np.histogram2d(*positions2D,bins=np.array([QBins,EnergyBins],dtype=object),weights=I.flatten())[0]
    MonitorCount=  np.histogram2d(*positions2D,bins=np.array([QBins,EnergyBins],dtype=object),weights=Monitor.flatten())[0]
    Normalization= np.histogram2d(*positions2D,bins=np.array([QBins,EnergyBins],dtype=object),weights=Norm.flatten())[0]

    # Calcualte the intensities
    Int = np.divide(intensity*normcounts,MonitorCount*Normalization)
    #eMean = 0.5*(EnergyBins[:-1]+EnergyBins[1:])
    #qMean = 0.5*(QBins[:-1]+QBins[1:])

    
    if plotPowderAverage: # Plot the powder average
        powderFig,powderAx = plt.subplots()
        X,Y = np.meshgrid(QBins,EnergyBins)
        powderAx.pcolormesh(X,Y,Int.T)
    
    if verbose:
        print('Finding intensities for background dataset')
    # Find intensites for individual points in the scan across data files
    for count,df in enumerate(newBackgroundDataSet):
        dfPosition = np.array([np.linalg.norm([df.qx,df.qy],axis=0),df.energy])
        # Calculate the bin index
        qIndex = np.floor((dfPosition[0]-QBins[0])/dQ).astype(int)#np.asarray([np.argmin(np.abs(qpos-qMean)) for qpos in dfPosition[0].flatten()]).reshape(*dfPosition[0].shape)
        eIndex = np.floor((dfPosition[1]-EnergyBins[0])/dE).astype(int)#np.asarray([np.argmin(np.abs(epos-eMean)) for epos in dfPosition[1].flatten()]).reshape(*dfPosition[1].shape)

        # Clamp the indicies (Might not be needed....)
        qIndex[qIndex<0]=0
        qIndex[qIndex>Int.shape[0]-1]=Int.shape[0]-1
        eIndex[eIndex<0]=0
        eIndex[eIndex>Int.shape[1]-1]=Int.shape[1]-1
        
        # Find intensity by rescaling with monitor and normalization
        df.I = Int[[qIndex],[eIndex]][0]*df.Monitor*df.Norm
        if verbose:
            print('df',count+1,'of',len(backgroundDataSet))
    # update the background data set
    newBackgroundDataSet._getData()
    # Apply the foreground mask to the background dataset
    
    if foregroundMask is None:
        newBackgroundDataSet.mask = foregroundDataSet.mask
    else:
        newBackgroundDataSet.mask = foregroundMask(newBackgroundDataSet)
    
    if plotPowderAverage:
        return newBackgroundDataSet,powderAx
    return newBackgroundDataSet
    

Reads in fitting functions

In [3]:
def gaussian(x,bg, amp1, cen1, wid1):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))
def gaussianslope(x,bg,slope, amp1, cen1, wid1):
    return bg+slope*(x-cen1)+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))
def gaussianslopefixedzero(x,bg,slope, amp1, wid1):
    cen1=0
    return bg+slope*(x-cen1)+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))
def gaussianexp(x,bg,alpha, amp1, cen1, wid1):
    return bg+np.exp(-alpha*(x-cen1))+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))
def gaussian2(x,bg, amp1, cen1, wid1, amp2, cen2, wid2):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))+amp2/(np.sqrt(2*np.pi)*wid2) * np.exp(-(x-cen2)**2 /(2*wid2**2))
def gaussian2slope(x,bg,slope, amp1, cen1, wid1, amp2, cen2, wid2):
    return bg+slope*(x-cen1)+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))+amp2/(np.sqrt(2*np.pi)*wid2) * np.exp(-(x-cen2)**2 /(2*wid2**2))
def gaussian3(x,bg, amp1, cen1, wid1, amp2, cen2, wid2, amp3, cen3, wid3):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))+amp2/(np.sqrt(2*np.pi)*wid2) * np.exp(-(x-cen2)**2 /(2*wid2**2))+amp3/(np.sqrt(2*np.pi)*wid3) * np.exp(-(x-cen3)**2 /(2*wid3**2))
def gaussian4(x,bg, amp1, cen1, wid1, amp2, cen2, wid2, amp3, cen3, wid3, amp4, cen4, wid4):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))+amp2/(np.sqrt(2*np.pi)*wid2) * np.exp(-(x-cen2)**2 /(2*wid2**2))+amp3/(np.sqrt(2*np.pi)*wid3) * np.exp(-(x-cen3)**2 /(2*wid3**2))+amp4/(np.sqrt(2*np.pi)*wid4) * np.exp(-(x-cen4)**2 /(2*wid4**2))

In [4]:
def cleaning(ds,treshold):
    i=0
    for dataset in range(len(ds)):
        for a3 in range(0,len(ds.a3[dataset])-1):
            if (np.sum(ds[dataset].I[a3,:,:],axis=(0,1))<treshold):
                ds[dataset].I[a3,:,:]=0
                ds[dataset].Monitor[a3,:,:]=0
                i=i+1
    return i

# Data

Example how to load and convert files

(choose binning 1-8)

Use binning 8 for high resolution

Load in dataset if you want to rotate by angle alpha

## H0L

In [5]:
files = _tools.fileListGenerator("1371-1388",r"U:\Data\CAMEA\MnF2\hdfH0L",2021)
dsH0L = DataSet.DataSet(files)
dsH0L.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsH0L,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsH0L):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L)) 

0


## 09.10.24

1.5 K, 0 T, 180 degrees

In [ ]:
files = _tools.fileListGenerator("2588-2608",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds = DataSet.DataSet(files)
ds.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds,treshold)
print(i)

## 10.10.24: Improving resolution

Original Setting

In [ ]:
files = _tools.fileListGenerator("2609-2612",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds1 = DataSet.DataSet(files)
ds1.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds1,treshold)
print(i)

Measurement

In [ ]:
files = _tools.fileListGenerator("2613-2616",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds2 = DataSet.DataSet(files)
ds2.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds2.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds2.mask,np.sum(mask)(ds2))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds2,treshold)
print(i)

Close sample slits (originally msb 21, mst 11)

msb 18

In [ ]:
files = _tools.fileListGenerator("2617-2620",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds3 = DataSet.DataSet(files)
ds3.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds3,treshold)
print(i)

msb 15

In [ ]:
files = _tools.fileListGenerator("2621-2624",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds4 = DataSet.DataSet(files)
ds4.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds4,treshold)
print(i)

msb 12

In [ ]:
files = _tools.fileListGenerator("2625-2628",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds5 = DataSet.DataSet(files)
ds5.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds5,treshold)
print(i)

msb 10

In [ ]:
files = _tools.fileListGenerator("2629-2632",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds6 = DataSet.DataSet(files)
ds6.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds6,treshold)
print(i)

## 10.10.24

1.5 K, 0 T, High E-resolution scan

In [ ]:
files = _tools.fileListGenerator("2644-2659",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds1 = DataSet.DataSet(files)
ds1.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds1.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds1.mask,np.sum(mask)(ds1))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds1,treshold)
print(i)

## 11.10.24

1.5 K, 10 T, 180 degrees

In [ ]:
files = _tools.fileListGenerator("2671-2694",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds = DataSet.DataSet(files)
ds.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds,treshold)
print(i)

1.5 K, 10 T, High E-resolution scan

In [ ]:
files = _tools.fileListGenerator("2695-2710",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds2 = DataSet.DataSet(files)
ds2.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds2.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds2.mask,np.sum(mask)(ds2))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds2,treshold)
print(i)

## 13.10.24

1.5 K, 10 T, FC High E-resolution scan

In [ ]:
files = _tools.fileListGenerator("2711-2718",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds = DataSet.DataSet(files)
ds.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds.mask,np.sum(mask)(ds))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds,treshold)
print(i)

## 14.10.24

120 degree, 1.5 K, 0 T, ZFC, (HK0)

In [ ]:
files = _tools.fileListGenerator("2745-2767",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds = DataSet.DataSet(files)
ds.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds,treshold)
print(i)

## 15.10.24

In [ ]:
files = _tools.fileListGenerator("2783-2799",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds = DataSet.DataSet(files)
ds.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds,treshold)
print(i)

# Masking, Absorption correction, symmetrization

In case some data should be masked out (needs modification for your sample) (adapt accordingly)

In [ ]:
mask=[]
ds=ds
#Mask for Currate Axes spurions (Q points in [] and width given in dqx, dqy)
mask.append(Mask.CurratAxeMask([[1,1,0],[-1,-1,0],[1,0,0]],dqx=0.07,dqy=0.07))

ds.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds.mask,np.sum(mask)(ds))]

Absorption correction

In [ ]:
##Files the absorption correction is based on
files = _tools.fileListGenerator("3059-3060,3072-3073,3084-3085",r"/Users/danielmazzone/Documents/CAMEA/2023/experiments/20222589/data",2023)
absorptionfile = DataSet.DataSet(files)
absorptionfile.convertDataFile(binning = 8,saveFile=False) 

##a3 range of correction
a3start=74
a3end=115
# found"background" of the region
meanI = 13200*3

## Extract A3 positions from first data file
a3 = absorptionfile[0].A3
 
rescale = np.ones_like(a3)
 
# Assuming that between a3 = a3start and a3 = a3end rescaling is to be performed
scaling = np.logical_and(a3>a3start,a3<a3end)
 
# Sum the elastic line in relevant tubes and for elastic energies
y = np.array([df.I[:,a3start:a3end,-8:].sum(axis=(1,2)) for df in absorptionfile])
 
# Add the intensities togehter, flipping second datafile due to the scan direction
y  = y[0] + y[1][::-1]+y[2] + y[3][::-1]+y[4] + y[5][::-1]
 
# Calculate the rescaling/a3 point
rescale[scaling] = (meanI/y)[scaling]
 
# Generate an interpolation function padded with 1 outside range
calcRescale = interpolate.interp1d(a3,rescale,fill_value=1.0,bounds_error=False)
if True: # make the plot of the rescaling
    fig,ax = plt.subplots()

    for df in absorptionfile[:6]:
        a3 = df.A3

        rescale = np.ones_like(a3)
        scaling = np.logical_and(a3>a3start,a3<a3end)
        rescale = calcRescale(df.A3)

        IRadial = df.I[:,a3start:a3end,-8:].sum(axis=(1,2))
        ax.scatter(a3,IRadial,label='Elastic Line',c='gray')
        ax.scatter(a3,IRadial*rescale,label='Rescaled '+df.name)

    ax.set_xlabel('A3 [deg]')
    ax.set_ylabel('Int [arb]')
    
if True: # perform rescaling
    for df in ds:
        rescale = calcRescale(df.A3[:df.I.shape[0]]).reshape(-1,1,1)
        df.Monitor/=rescale.astype(float)

Symmetrization (adapt accordingly)

In [ ]:
for i,df in enumerate(ds):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

# Powder average bg subtraction (adapt accordingly)

In [ ]:
bgdata = _tools.fileListGenerator("1475-1512",r"/Users/danielmazzone/Documents/CAMEA/2023/experiments/20222570/data",2023)
#bgdata = _tools.fileListGenerator("1475-1478,1491-1498,1503-1506",r"/Users/danielmazzone/Documents/CAMEA/2023/experiments/20222570/data",2023)
bgdata = DataSet.DataSet(bgdata)

bgdata.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(bgdata,treshold)
print(i)

In [ ]:
BGmask=[]
bgdata=bgdata
#Mask for Currate Axes spurions (Q points in [] and width given in dqx, dqy)
mask.append(Mask.CurratAxeMask([[1,1,0],[1,-1,0],[-1,1,0],[-1,-1,0]],dqx=0.1,dqy=0.1))
mask.append(Mask.CurratAxeMask([[1,0,0],[0,1,0],[-1,0,0],[0,-1,0]],dqx=0.07,dqy=0.07))

#Fwhm of 4.5 meV dataset is 0.2 meV
#Will cut away everything above *FWHM=0.3 for 4.5 meV dataset
BGmask.append(Mask.lineMask(4.2,4.8,coordinates='Ei',maskInside=True)*Mask.lineMask(0.3,10,coordinates='energy',maskInside=False))

#Maskin gfluctuations
BGmask.append(Mask.circleMask([1,1],[1.4,1],coordinates=['h','k'],maskInside=True))
BGmask.append(Mask.circleMask([-1,1],[-1.4,1],coordinates=['h','k'],maskInside=True))
BGmask.append(Mask.circleMask([1,-1],[1.4,-1],coordinates=['h','k'],maskInside=True))
BGmask.append(Mask.circleMask([1,0],[1.4,0],coordinates=['h','k'],maskInside=True))
BGmask.append(Mask.circleMask([0,1],[0.4,1],coordinates=['h','k'],maskInside=True))
bgdata.mask = [np.logical_or(m1,m2) for m1,m2 in zip(bgdata.mask,np.sum(BGmask)(bgdata))]

In [ ]:
bgDS,ax = generateBG(ds,bgdata,dQ=0.02,dE=0.03,plotPowderAverage=True,verbose=True)

sub = ds-bgDS

# Plotting Examples

3D view of datase ds

(binning along Qx, Qy, energy)

Qx nd Qx = 0.02 are good if 1 degree steps in a3 are used, energy 0.1-0.2 is reasonable for overview

min and max of c-axis

In [ ]:
Viewer = dsHR.View3D(0.03,0.03,.1,grid=9,rlu=True)
Viewer.caxis=(-0.0,3)
plt.show()

In [ ]:
Viewer.caxis=(0.0,0.1)

Checks length of BZ

This is used to estimated width and min pixels later

In [ ]:
print('BZ length H in AA-1 = '+str(2*np.pi/ds[0].sample.a))
print('BZ length K in AA-1 = '+str(2*np.pi/ds[0].sample.b))
print('BZ length L in AA-1 = '+str(2*np.pi/ds[0].sample.c))

Creates a powder average of the data

In [ ]:
ds=ds
EMin = 0  #Energy minimum
EMax = 4 #Energy maximum
dE = .1 #Energy steps
QMin =0 #1/A
QMax =2.5 #1/A
dQ=.02 #Q steps

vmin=0 #minimum of the colorscale
vmax=0.05 #maximum of colorscale
cmap=parula
data=ds.plotCutPowder(EBins=np.arange(EMin,EMax,dE), QBins=np.arange(QMin,QMax, dQ),vmin=vmin,vmax=vmax)
fig,axs3=plt.subplots(1,figsize=(5,5))
axs3.errorbar(np.array(data[1]['Energy']),np.array(data[1]['Int']),np.array(data[1]['Int_err']), marker='o',color=[1,0.4,0],linestyle='')


Example how to plot a Q-Plane over an energy window

Plots unsmoothed and smoothed Q-Plane

In [ ]:
EMin =6.0 #Energy minimum
EMax =7.8 #Energy maximum
#xBinTolerance = 2*np.pi/ds[0].sample.c*.03 #value in rlu. binning along x
#yBinTolerance = np.sqrt(5)*2*np.pi/ds[0].sample.b*0.03 #value in rlu
xBinTolerance = .03 #value in 1/A. binning along x
yBinTolerance = .03 #value in 1/A. binning along y
vmin=0.0 #minimum of the color scale
vmax=0.8 #maximum of the color scale
#cmap=parula #cmap for smoothed plane
#cmap='jet'
Data,ax = ds.plotQPlane(EMin=EMin, EMax=EMax,xBinTolerance=xBinTolerance,yBinTolerance=yBinTolerance,log=False,cmap = 'turbo')
fig = ax.get_figure() # Extract figure from returned axis
fig.colorbar(ax.pmeshs[0]) # Create colorbar from plot
ax.set_clim(vmin,vmax)

Example how to plot QE-Plane

In [ ]:
dset=ds
rlu = True
#grid = -10
grid = False
self = dset
dataFiles = None
rlu = True
Q1 = np.array([0.5,0.5,0.5]) #Q1 of cut
Q2 = np.array([1,1,0.5])  #Q2 of cut
#width = 2*np.pi/ds[0].sample.a/(2**(1/2))*.1 #integration width perpendicular to Q
#minPixel = 2*np.pi/ds[0].sample.c*0.01 #step along Q
EMin = 6.4 #Energy minimum
EMax = 7.3 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.025 #integration width perpendicular to Q in 1/A
minPixel = 0.025 #step along Q in 1/A
vmin=0 #caxis minimum
vmax=0.7 #caxis maximum
#cmap='viridis' #cmap for coloscale
#cmap=parula
#cmap='Reds'
axSUB,data,bins = dset.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=True,cmap='turbo')
#axSUB.set_title('title')
#######################

Subtract two QE slices

In [ ]:
dset1=ds1
dset2=ds2
rlu = True
#grid = -10
grid = False
self1 = dset1
self2=dset2
dataFiles = None
rlu = True
Q1 = np.array([0.5,0.5,0.5]) #Q1 of cut
Q2 = np.array([1,1,0.5])  #Q2 of cut
#width = 2*np.pi/ds[0].sample.a/(2**(1/2))*.1 #integration width perpendicular to Q
#minPixel = 2*np.pi/ds[0].sample.c*0.01 #step along Q
EMin = 6.4 #Energy minimum
EMax = 7.3 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.025 #integration width perpendicular to Q in 1/A
minPixel = 0.025 #step along Q in 1/A
vmin=0 #caxis minimum
vmax=0.7 #caxis maximum
#cmap='viridis' #cmap for coloscale
#cmap=parula
#cmap='Reds'
axSUB1,data1,bins1 = dset1.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=True,cmap='turbo')
axSUB2,data2,bins2 = dset2.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=True,cmap='turbo')
dset1m2=data1-data2
axSUB1m2,data1m2,bins1m2 = dset1m2.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=True,cmap='turbo')
#axSUB.set_title('title')
#######################

In [ ]:
center=-1
minPixelr_rlu = minPixel/2/np.pi*ds[0].sample.a
for H, dat in data[np.logical_and(data['L']<=center+minPixelr_rlu/2,data['L']>center-minPixelr_rlu/2)].groupby('L'):
    print(H)
fig,axs2=plt.subplots(1,figsize=(5,5))
axs2.errorbar(dat['Energy'],dat['Int'],dat['Int_err'], marker='o',color=[.8,0.6,0],linestyle='',label=r'Q='+str(center))
#gmodel = Model(gaussian2)
#result = gmodel.fit(dat['Int'], x=dat['Energy'], bg=0.001, amp1=1,cen1=2.4, wid1=0.1, amp2=1,cen2=2.75, wid2=0.1)
#axs2.plot(np.linspace(np.array(dat['Energy'])[0],np.array(dat['Energy'])[-1],1000),
#gaussian2(np.linspace(np.array(dat['Energy'])[0],np.array(dat['Energy'])[-1],1000),result.params['bg'],
#result.params['amp1'],result.params['cen1'],result.params['wid1'],
#result.params['amp2'],result.params['cen2'],result.params['wid2']),color=[1,0.4,0],linestyle='--')

gmodel = Model(gaussianslope)
result = gmodel.fit(dat['Int'], x=dat['Energy'], bg=0.001,slope=0, amp1=1,cen1=2.5, wid1=0.1)
axs2.plot(np.linspace(np.array(dat['Energy'])[0],np.array(dat['Energy'])[-1],1000),
gaussianslope(np.linspace(np.array(dat['Energy'])[0],np.array(dat['Energy'])[-1],1000),result.params['bg'],result.params['slope'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),color=[1,0.4,1],linestyle='--')



axs2.set_ylabel('I (a.u.)')
axs2.set_xlabel('E = (meV)')
axs2.legend(frameon=False)
print(result.params['cen1'])
print(result.params['wid1'])

print(result.params['cen2'])
print(result.params['wid2'])

Q-cuts

In [ ]:
Q1 = np.array([-1,-1,1])  #Q-position
Q2 = np.array([1,1,1])  #Q-position
scanvar='H' #define scan dirrection H, K, L
width = 2*np.pi/ds[0].sample.c*.05 #integration width perpendicular to Q in 1/A
minPixel = 2*np.pi/(ds[0].sample.a/(2**(1/2)))*0.025 #step along Q in rlu
#width= 2*np.pi/ds[0].sample.c*0.1 #integration width perpendicular to Q
#minPixel = 2*np.pi/(ds[0].sample.a)*0.01 #step along Q
EMin=-.2#Energy window
EMax=.2 #Energy window
rlu = True
constantBins = True
####Plots the cut####
data, bin = ds.cut1D(Emin=EMin,Emax=EMax, q1=Q1, q2=Q2,width=width,minPixel=minPixel,rlu=True,constantBins=constantBins,ufit=False,extend=False)

fig,axs2=plt.subplots(1,figsize=(5,5))
axs2.errorbar(data[scanvar],data['Int'],data['Int_err'], marker='o',color=[1,0.4,0],linestyle='-',label=r'Q='+str(Q1))

####Fits the cut with Model and start values and then plots it####
gmodel = Model(gaussian2)
result = gmodel.fit(I, x=K, bg=0.00, amp1=1, cen1=.5, wid1=.05, amp2=1, cen2=1.5, wid2=.05)
axs2.plot(np.linspace(data[scanvar][1],data[scanvar][end-1],1000),
gaussian2(np.linspace(data[scanvar][1],data[scanvar][end-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1'],     
result.params['amp2'],result.params['cen2'],result.params['wid2']),color=[0,0.4,1],linestyle='--')

#gmodel = Model(gaussianslope)
#result = gmodel.fit(I, x=K, bg=0.00, slope=0, amp1=1, cen1=-0.4, wid1=.05)
#axs2.plot(np.linspace(K[1],K[end-1],1000),
#gaussianslope(np.linspace(K[1],K[end-1],1000),result.params['bg'],result.params['slope'],
#result.params['amp1'],result.params['cen1'],result.params['wid1']),color=[0,0.4,1],linestyle='--')
print(result.params['cen1'])
print(result.params['cen2'])
######Makes title and axes
Eav=np.round(EMin+(EMax-EMin)/2,2)
Edif=np.int(np.round(100*(EMax-EMin)/2,0))
if scanvar!='Energy':
    axs2.set_title(r"$\Delta$E = "+  str(Eav) + "(" + str(Edif) + ") mev. Q$_1$ = " + str(Q1) + ', Q$_2$ =' +str(Q2))
    axs2.set_xlabel('q = '+str((Q1-Q2)/np.sqrt(np.sum((Q1-Q2)**2)))+' (r.l.u.)')
if scanvar=='Energy':
    axs2.set_title("q = "+  str(Q1) + "(" + str((Q1-Q2)/2) + ") (r.l.u.)")
    axs2.set_xlabel('E = (meV)')
#print(result.params['bg'])
#print(result.params['amp1']/(np.sqrt(2*np.pi)*result.params['wid1']))
#print((result.params['amp1']/(np.sqrt(2*np.pi)*result.params['wid1']))/result.params['bg']*100)

### Q cut to check alignment

In [16]:
ds = dsH0L
Q1 = np.array([0.8,0,0])  #Q-position
Q2 = np.array([1.2,0,0])  #Q-position
scanvar='H' #define scan dirrection H, K, L
width = 0.02 #integration width perpendicular to Q in 1/A
minPixel = 0.02 #step along Q in 1/A
EMin = 1.95 #Energy window
EMax = 2.05 #Energy window
rlu = True
constantBins = True
####Plots the cut####
data, bin = ds.cut1D(Emin=EMin,Emax=EMax, q1=Q1, q2=Q2,width=width,minPixel=minPixel,rlu=True,constantBins=constantBins,ufit=False,extend=False)

fig,axs2=plt.subplots(1,figsize=(5,5))
axs2.errorbar(data[scanvar],data['Int'],data['Int_err'], marker='o',color=[1,0.4,0],linestyle='-',label=r'Q='+str(Q1))

I = data['Int']
H = data['H']
####Fits the cut with Model and start values and then plots it####
gmodel = Model(gaussian2)
#gmodel = Model(gaussian)
result = gmodel.fit(I, x=H, bg=0.0001, amp1=1, cen1=0.9, wid1=.05, amp2=1, cen2=1.1, wid2=.05)
#result = gmodel.fit(I, x=H, bg=0.0001, amp1=1, cen1=1, wid1=.05)
axs2.plot(np.linspace(data[scanvar][1],data[scanvar][len(data[scanvar])-1],1000), gaussian2(np.linspace(data[scanvar][1],data[scanvar][len(data[scanvar])-1],1000), result.params['bg'], result.params['amp1'], result.params['cen1'], result.params['wid1'], result.params['amp2'], result.params['cen2'], result.params['wid2']), color=[0,0.4,1], linestyle='--')
#gmodel = Model(gaussian)
#result = gmodel.fit(I, x=H, bg=0.0001, amp1=1, cen1=1, wid1=.05)
#axs2.plot(np.linspace(data[scanvar][1],data[scanvar][len(data[scanvar])-1],1000), gaussian(np.linspace(data[scanvar][1],data[scanvar][len(data[scanvar])-1],1000), result.params['bg'], result.params['amp1'], result.params['cen1'], result.params['wid1']), color=[0,0.4,1], linestyle='--')
axs2.set_xlabel('H [rlu]')
axs2.set_ylabel('Intensity [Arb]')

#gmodel = Model(gaussianslope)
#result = gmodel.fit(I, x=K, bg=0.00, slope=0, amp1=1, cen1=-0.4, wid1=.05)
#axs2.plot(np.linspace(K[1],K[end-1],1000),
#gaussianslope(np.linspace(K[1],K[end-1],1000),result.params['bg'],result.params['slope'],
#result.params['amp1'],result.params['cen1'],result.params['wid1']),color=[0,0.4,1],linestyle='--')
print(result.params['cen1'])
print(result.params['cen2'])

<Parameter 'cen1', value=0.9290050331906032 +/- 0.00148, bounds=[-inf:inf]>
<Parameter 'cen2', value=1.0884137755796477 +/- 0.00147, bounds=[-inf:inf]>


General cutting path

In [ ]:
grid = False
dataFiles = None
rlu = True
dss=ds
#Q1 = np.array([0.5,0.5,0]) #Q1 of cut
#Q2 = np.array([0,0,1])  #Q2 of cut
#Q3 = np.array([1,1,0]) #Q1 of cu  
#Q4 = np.array([1,1,1]) #Q1 of cu 
#Q5 = np.array([0,0,1]) #Q1 of cu 
#Q1 = np.array([0,0,1]) #Q1 of cut
#Q2 = np.array([1,1,1])  #Q2 of cut
#Q3 = np.array([1,1,0.5]) #Q1 of cu  
#Q4 = np.array([0.5,0.5,0.5]) #Q1 of cu 
#Q5 = np.array([1,1,0]) #Q1 of cu 
Q1 = np.array([0.5,0.5,0.5]) #Q1 of cut
Q2 = np.array([1,1,0.5])  #Q2 of cut
EMin = 6.6  #Energy minimum
EMax = 7.2 #Energy maximum
dE = 0.04 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.03 #integration width perpendicular to Q in rlu
minPixel = 0.02 #step along Q in rlu
#width = 2*np.pi/ds[0].sample.c*.5 #integration width perpendicular to Q in 1/A
#minPixel = np.sqrt(2)*2*np.pi/ds[0].sample.a*0.01 #step along Q   in 1/A


vmin=0.0   #caxis minimum
vmax=1.0 #caxis maximum
cmap='turbo'
axFG,data,bins = dss.plotCutQELine(QPoints=np.array([Q1,Q2]),EnergyBins=energy,width=width,minPixel=minPixel,rlu=rlu,plotSeperator=True,seperatorWidth=1,seperatorColor='k',cmap=cmap)
axFG.set_title('1.5 K, 0 T, ZFC')
axFG.locator_params(axis='x', nbins=15)
axFG.set_ylabel(r'$\Delta$E [meV]',fontsize=16)
axFG.tick_params(axis='both', which='major', labelsize=16)
axFG.set_clim(vmin,vmax)

Example how to plot along a general path

1d-cu along energy (circular integration width)

In [ ]:
dss=ds
Q1 = np.array([1,1,0.5])  #Q-position

width = 0.025 #integration width perpendicular to Q in 1/A
minPixel = 0.05 #step along Energy
EMin=6.4 #Minimum of energy
EMax=7.4 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
data, bin = dss.cut1DE(E1=EMin,E2=EMax, q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
fig,axs3=plt.subplots(1,figsize=(5,5))
axs3.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='')
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.8, wid1=0.1)
axs3.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']), linestyle='--')

######Makes title and axes
Eav=np.round(EMin+(EMax-EMin)/2,1)
Edif=int(np.round(10*(EMax-EMin)/2,0))
if scanvar!='Energy':
    axs3.set_title(r"$\Delta$E = "+  str(Eav) + "(" + str(Edif) + ") mev")
    axs3.set_title(r"$\Delta$E = "+  str(Eav) + "(" + str(Edif) + ") mev. Q$_1$ = " + str(Q1) + ', Q$_2$ =' +str(Q2))
if scanvar=='Energy':
    axs3.set_title(r"Q: " + str(Q1) + ". FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
    #axs3.set_title(r"Q: " + str(Q1) + ". Center 1: " + str(round(result.params['cen1'].value, 3)) + " meV.")
    #axs3.set_title(r"Q: " + str(Q1) + ". Center 1: " + str(round(result.params['cen1'].value, 3)) + " meV. Center 2: " + str(round(result.params['cen2'].value, 3)) + " meV.")
    #axs3.set_title(r"Q: " + str(Q1))
    axs3.set_xlabel('E (meV)')
#print(result.chisqr)
axs3.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 2, linestyle = '-', color = 'k')
#axs3.legend(frameon=True)
axs3.set_ylabel(r'I (abs)')

# Load and integrate Ei=4.0 meV elastic line while closing slits

In [ ]:
import matplotlib.pyplot as plt
try:
    import IPython
    shell = IPython.get_ipython()
    shell.enable_matplotlib(gui='qt')
except:
    pass

from MJOLNIR import _tools
from MJOLNIR.Data import DataSet
from lmfit import Model, Parameter, report_fit
from MJOLNIR.Data import Mask
from MJOLNIR._tools import fileListGenerator

import numpy as np
from os import path
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd
import copy

In [ ]:
def gaussian(x, bg, amp1, cen1, wid1):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))

In [ ]:
def cleaning(ds,treshold):
    i=0
    for dataset in range(len(ds)):
        for a3 in range(0,len(ds.a3[dataset])-1):
            if (np.sum(ds[dataset].I[a3,:,:],axis=(0,1))<treshold):
                ds[dataset].I[a3,:,:]=0
                ds[dataset].Monitor[a3,:,:]=0
                i=i+1
    return i

## Import data: 10.10.24 Improving resolution

In [ ]:
# Original setting: bottom slit = 21 mm, top slit = 11 mm, virtual source slits = 40 mm
files = _tools.fileListGenerator("2609-2612",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds1 = DataSet.DataSet(files)
ds1.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds1,treshold)
print(i)

# bottom slit = 21 mm, top slit = 11 mm, virtual source slits closed to 10 mm
files = _tools.fileListGenerator("2613-2616",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds2 = DataSet.DataSet(files)
ds2.convertDataFile(binning = 8,saveFile=False) 
# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds2.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds2.mask,np.sum(mask)(ds2))]
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds2,treshold)
print(i)

# bottom slit = 18 mm, top slit = 8 mm, virtual sample slits closed to 10 mm
files = _tools.fileListGenerator("2617-2620",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds3 = DataSet.DataSet(files)
ds3.convertDataFile(binning = 8,saveFile=False) 
# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds3.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds3.mask,np.sum(mask)(ds3))]
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds3,treshold)
print(i)

# bottom slit = 15 mm, top slit = 5 mm, virtual sample slits closed to 10 mm
files = _tools.fileListGenerator("2621-2624",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds4 = DataSet.DataSet(files)
ds4.convertDataFile(binning = 8,saveFile=False) 
# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds4.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds4.mask,np.sum(mask)(ds4))]
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds4,treshold)
print(i)

# bottom slit = 12 mm, top slit = 2 mm, virtual sample slits closed to 10 mm
files = _tools.fileListGenerator("2625-2628",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds5 = DataSet.DataSet(files)
ds5.convertDataFile(binning = 8,saveFile=False) 
# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds5.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds5.mask,np.sum(mask)(ds5))]
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds5,treshold)
print(i)

# bottom slit = 10 mm, top slit = 0 mm, virtual sample slits closed to 10 mm
files = _tools.fileListGenerator("2629-2632",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds6 = DataSet.DataSet(files)
ds6.convertDataFile(binning = 8,saveFile=False) 
# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds6.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds6.mask,np.sum(mask)(ds6))]
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds6,treshold)
print(i)

## Integrate elastic line

In [ ]:
Q = np.array([0.5,0.5,0.5])  #Q-position

width = 100 #integration width perpendicular to Q in 1/A
minPixel = 0.05 #step along Energy
EMin = -0.5 #Minimum of energy
EMax = 0.5 #Maximum of enerrgy
rlu = True
constantBins = True

# bottom slit = 21 mm, top slit = 11 mm, virtual source slits = 40 mm
data1, bin1 = ds1.cut1DE(E1=EMin, E2=EMax, q=Q, width=width, minPixel=minPixel, rlu=rlu, constantBins=constantBins, ufit=False)
fig1, axs1 = plt.subplots(1,figsize=(5,5))
axs1.errorbar(data1['Energy'], data1['Int'], data1['Int_err'], marker='o', linestyle='')
gmodel = Model(gaussian)
result = gmodel.fit(data1['Int'], x=data1['Energy'], bg=0.001, amp1=1, cen1=0, wid1=0.1)
axs1.plot(np.linspace(np.array(data1['Energy'])[0], np.array(data1['Energy'])[-1], 1000),
gaussian(np.linspace(np.array(data1['Energy'])[0], np.array(data1['Energy'])[-1], 1000), result.params['bg'],
result.params['amp1'], result.params['cen1'], result.params['wid1']), linestyle='--')
fwhm1 = result.params['wid1'].value*2*np.sqrt(2*np.log(2))
fwhmErr1 = result.params['wid1'].stderr*2*np.sqrt(2*np.log(2))
vSlit1 = 32 # Vertical slit width
axs1.set_title(r"Q: " + str(Q) + ". FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
axs1.set_xlabel('E (meV)')
axs1.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 2, linestyle = '-', color = 'k')
axs1.set_ylabel(r'I (abs)')

print('Ei = 4.0 meV elastic line FWHM before closing slits: ' + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

# Closed virtual source slits to 10 mm
# bottom slit = 21 mm, top slit = 11 mm
data2, bin2 = ds2.cut1DE(E1=EMin, E2=EMax, q=Q, width=width, minPixel=minPixel, rlu=rlu, constantBins=constantBins, ufit=False)
fig2, axs2 = plt.subplots(1,figsize=(5,5))
axs2.errorbar(data2['Energy'], data2['Int'], data2['Int_err'], marker='o', linestyle='')
gmodel = Model(gaussian)
result = gmodel.fit(data2['Int'], x=data2['Energy'], bg=0.001, amp1=1, cen1=0, wid1=0.1)
axs2.plot(np.linspace(np.array(data2['Energy'])[0], np.array(data2['Energy'])[-1], 1000),
gaussian(np.linspace(np.array(data2['Energy'])[0], np.array(data2['Energy'])[-1], 1000), result.params['bg'],
result.params['amp1'], result.params['cen1'], result.params['wid1']), linestyle='--')
fwhm2 = result.params['wid1'].value*2*np.sqrt(2*np.log(2))
fwhmErr2 = result.params['wid1'].stderr*2*np.sqrt(2*np.log(2))
vSlit2 = 32 # Vertical slit width
axs2.set_title(r"Q: " + str(Q) + ". FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
axs2.set_xlabel('E (meV)')
axs2.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 2, linestyle = '-', color = 'k')
axs2.set_ylabel(r'I (abs)')

print('Ei = 4.0 meV elastic line FWHM after closing virtual slits: ' + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

# bottom slit = 18 mm, top slit = 8 mm
data3, bin3 = ds3.cut1DE(E1=EMin, E2=EMax, q=Q, width=width, minPixel=minPixel, rlu=rlu, constantBins=constantBins, ufit=False)
fig3, axs3 = plt.subplots(1,figsize=(5,5))
axs3.errorbar(data3['Energy'], data3['Int'], data3['Int_err'], marker='o', linestyle='')
gmodel = Model(gaussian)
result = gmodel.fit(data3['Int'], x=data3['Energy'], bg=0.001, amp1=1, cen1=0, wid1=0.1)
axs3.plot(np.linspace(np.array(data3['Energy'])[0], np.array(data3['Energy'])[-1], 1000),
gaussian(np.linspace(np.array(data3['Energy'])[0], np.array(data3['Energy'])[-1], 1000), result.params['bg'],
result.params['amp1'], result.params['cen1'], result.params['wid1']), linestyle='--')
fwhm3 = result.params['wid1'].value*2*np.sqrt(2*np.log(2))
fwhmErr3 = result.params['wid1'].stderr*2*np.sqrt(2*np.log(2))
vSlit3 = 26 # Vertical slit width
axs3.set_title(r"Q: " + str(Q) + ". FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
axs3.set_xlabel('E (meV)')
axs3.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 2, linestyle = '-', color = 'k')
axs3.set_ylabel(r'I (abs)')

# bottom slit = 15 mm, top slit = 5 mm
data4, bin4 = ds4.cut1DE(E1=EMin, E2=EMax, q=Q, width=width, minPixel=minPixel, rlu=rlu, constantBins=constantBins, ufit=False)
fig4, axs4 = plt.subplots(1,figsize=(5,5))
axs4.errorbar(data4['Energy'], data4['Int'], data4['Int_err'], marker='o', linestyle='')
gmodel = Model(gaussian)
result = gmodel.fit(data4['Int'], x=data4['Energy'], bg=0.001, amp1=1, cen1=0, wid1=0.1)
axs4.plot(np.linspace(np.array(data4['Energy'])[0], np.array(data4['Energy'])[-1], 1000),
gaussian(np.linspace(np.array(data4['Energy'])[0], np.array(data4['Energy'])[-1], 1000), result.params['bg'],
result.params['amp1'], result.params['cen1'], result.params['wid1']), linestyle='--')
fwhm4 = result.params['wid1'].value*2*np.sqrt(2*np.log(2))
fwhmErr4 = result.params['wid1'].stderr*2*np.sqrt(2*np.log(2))
vSlit4 = 20 # Vertical slit width
axs4.set_title(r"Q: " + str(Q) + ". FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
axs4.set_xlabel('E (meV)')
axs4.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 2, linestyle = '-', color = 'k')
axs4.set_ylabel(r'I (abs)')

# bottom slit = 12 mm, top slit = 2 mm
data5, bin5 = ds5.cut1DE(E1=EMin, E2=EMax, q=Q, width=width, minPixel=minPixel, rlu=rlu, constantBins=constantBins, ufit=False)
fig5, axs5 = plt.subplots(1,figsize=(5,5))
axs5.errorbar(data5['Energy'], data5['Int'], data5['Int_err'], marker='o', linestyle='')
gmodel = Model(gaussian)
result = gmodel.fit(data5['Int'], x=data5['Energy'], bg=0.001, amp1=1, cen1=0, wid1=0.1)
axs5.plot(np.linspace(np.array(data5['Energy'])[0], np.array(data5['Energy'])[-1], 1000),
gaussian(np.linspace(np.array(data5['Energy'])[0], np.array(data5['Energy'])[-1], 1000), result.params['bg'],
result.params['amp1'], result.params['cen1'], result.params['wid1']), linestyle='--')
fwhm5 = result.params['wid1'].value*2*np.sqrt(2*np.log(2))
fwhmErr5 = result.params['wid1'].stderr*2*np.sqrt(2*np.log(2))
vSlit5 = 14 # Vertical slit width
axs5.set_title(r"Q: " + str(Q) + ". FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
axs5.set_xlabel('E (meV)')
axs5.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 2, linestyle = '-', color = 'k')
axs5.set_ylabel(r'I (abs)')

# bottom slit = 10 mm, top slit = 0 mm
data6, bin6 = ds6.cut1DE(E1=EMin, E2=EMax, q=Q, width=width, minPixel=minPixel, rlu=rlu, constantBins=constantBins, ufit=False)
fig6, axs6 = plt.subplots(1,figsize=(5,5))
axs6.errorbar(data6['Energy'], data6['Int'], data6['Int_err'], marker='o', linestyle='')
gmodel = Model(gaussian)
result = gmodel.fit(data6['Int'], x=data6['Energy'], bg=0.001, amp1=1, cen1=0, wid1=0.1)
axs6.plot(np.linspace(np.array(data6['Energy'])[0], np.array(data6['Energy'])[-1], 1000),
gaussian(np.linspace(np.array(data6['Energy'])[0], np.array(data6['Energy'])[-1], 1000), result.params['bg'],
result.params['amp1'], result.params['cen1'], result.params['wid1']), linestyle='--')
fwhm6 = result.params['wid1'].value*2*np.sqrt(2*np.log(2))
fwhmErr6 = result.params['wid1'].stderr*2*np.sqrt(2*np.log(2))
vSlit6 = 10 # Vertical slit width
axs6.set_title(r"Q: " + str(Q) + ". FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
axs6.set_xlabel('E (meV)')
axs6.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 2, linestyle = '-', color = 'k')
axs6.set_ylabel(r'I (abs)')

print('Ei = 4.0 meV elastic line FWHM after closing slits: ' + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

vSlit = [vSlit1, vSlit2, vSlit3, vSlit4, vSlit5, vSlit6]
fwhm = [fwhm1, fwhm2, fwhm3, fwhm4, fwhm5, fwhm6]
fwhmErr = [fwhmErr1, fwhmErr2, fwhmErr3, fwhmErr4, fwhmErr5, fwhmErr6]
fig, axs = plt.subplots(1,figsize=(5,5))
axs.errorbar(vSlit, fwhm, fwhmErr, marker='o', linestyle='', markersize='10', capsize=3, elinewidth=3)

## Cut through HHL 0 T global scan 6.9 meV mode

In [ ]:
files = _tools.fileListGenerator("2588-2608",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHHL = DataSet.DataSet(files)
dsHHL.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHHL,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsHHL):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

In [ ]:
Q = np.array([0.5,0.5,0.5])  #Q-position

width = 0.05 # integration width perpendicular to Q in 1/A
minPixel = 0.08 # step along Energy
EMin = 6.0 # Minimum of energy
EMax = 8.0 # Maximum of enerrgy
rlu = True
constantBins = True

dataHHL, binHHL = dsHHL.cut1DE(E1=EMin, E2=EMax, q=Q, width=width, minPixel=minPixel, rlu=rlu, constantBins=constantBins, ufit=False)
figHHL, axsHHL = plt.subplots(1,figsize=(5,5))
axsHHL.errorbar(dataHHL['Energy'], dataHHL['Int'], dataHHL['Int_err'], marker='o', linestyle='')
gmodel = Model(gaussian)
result = gmodel.fit(dataHHL['Int'], x=dataHHL['Energy'], bg=0.001, amp1=1, cen1=7.0, wid1=0.1)
axsHHL.plot(np.linspace(np.array(dataHHL['Energy'])[0], np.array(dataHHL['Energy'])[-1], 1000),
gaussian(np.linspace(np.array(dataHHL['Energy'])[0], np.array(dataHHL['Energy'])[-1], 1000), result.params['bg'],
result.params['amp1'], result.params['cen1'], result.params['wid1']), linestyle='--')
fwhmHHL = result.params['wid1'].value*2*np.sqrt(2*np.log(2))
fwhmErrHHL = result.params['wid1'].stderr*2*np.sqrt(2*np.log(2))
axsHHL.set_title(r"Q: " + str(Q) + ". FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
axsHHL.set_xlabel('E (meV)')
axsHHL.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 2, linestyle = '-', color = 'k')
axsHHL.set_ylabel(r'I (abs)')

print('Ei = 11.5 meV, Ef = 4.6 meV inelastic mode at (1/2,1/2,1/2) FWHM before closing slits: ' + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + "+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

# Look for dipole-dipole splitting from M to A

## Import libraries and define functions

In [ ]:
import matplotlib.pyplot as plt
try:
    import IPython
    shell = IPython.get_ipython()
    shell.enable_matplotlib(gui='qt')
except:
    pass

from MJOLNIR import _tools
from MJOLNIR.Data import DataSet
from lmfit import Model, Parameter, report_fit
from MJOLNIR.Data import Mask
from MJOLNIR._tools import fileListGenerator

import numpy as np
from os import path
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd
import copy

def gaussian(x, bg, amp1, cen1, wid1):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))

def cleaning(ds,treshold):
    i=0
    for dataset in range(len(ds)):
        for a3 in range(0,len(ds.a3[dataset])-1):
            if (np.sum(ds[dataset].I[a3,:,:],axis=(0,1))<treshold):
                ds[dataset].I[a3,:,:]=0
                ds[dataset].Monitor[a3,:,:]=0
                i=i+1
    return i

## Import and fit to low-resolution (H0L) data

In [ ]:
# Import H0L data
files = _tools.fileListGenerator("1371-1388",r"U:\Data\CAMEA\MnF2\hdfH0L",2021)
dsH0L = DataSet.DataSet(files)
dsH0L.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsH0L,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsH0L):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L)) 

In [ ]:
# Plot QE slice
%matplotlib qt
dset = dsH0L
rlu = True
#grid = -10
grid = False
self = dset
dataFiles = None
rlu = True
Q1 = np.array([1,0,0]) #Q1 of cut
Q2 = np.array([0.5,0,0.5]) #Q2 of cut
EMin = 0 #Energy minimum
EMax = 8 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.04 #integration width perpendicular to Q in 1/A
minPixel = 0.03 #step along Q in 1/A
vmin = 0 #caxis minimum
vmax = 0.1 #caxis maximum

ax,data,bins = dset.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=True,cmap='turbo')
ax.set_ylabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=16)
ax.set_xlabel("")
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_clim(vmin, vmax)
ax.set_xlim(0, 1)
ax.get_figure().set_size_inches(10,5)
ax.set_xticks([0, 1], (r"$\Gamma$", "R"))
cb = ax.colorbar
cb.set_label(label='$I$ (arb. units)', size=16)
cb.ax.tick_params(labelsize=16)
cb.set_ticks([vmin, vmax])
plt.show()

## Import and fit to low-resolution (HHL) data

In [ ]:
# Import HHL data
files = _tools.fileListGenerator("2588-2608",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHHL = DataSet.DataSet(files)
dsHHL.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHHL,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsHHL):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

In [ ]:
# Plot QE slice
%matplotlib qt
dset = dsHHL
rlu = True
#grid = -10
grid = False
self = dset
dataFiles = None
rlu = True
Q1 = np.array([0.5,0.5,1]) #Q1 of cut
Q2 = np.array([0.5,0.5,0.5]) #Q2 of cut
EMin = 0 #Energy minimum
EMax = 8 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.04 #integration width perpendicular to Q in 1/A
minPixel = 0.02 #step along Q in 1/A
vmin = 0 #caxis minimum
vmax = 1 #caxis maximum

ax,data,bins = dset.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=True,cmap='turbo')
ax.set_ylabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=16)
ax.set_xlabel("")
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_clim(vmin, vmax)
ax.set_xlim(0, 1)
ax.get_figure().set_size_inches(10,5)
ax.set_xticks([0, 1], ("M", "A"))
cb = ax.colorbar
cb.set_label(label='$I$ (arb. units)', size=16)
cb.ax.tick_params(labelsize=16)
cb.set_ticks([vmin, vmax])
plt.show()

In [ ]:
# Cut through M
Q1 = np.array([0.5,0.5,1])  #Q-position

width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin = 5 #Minimum of energy
EMax = 6.8 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dset.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)

# Fit to a Gaussian and plot
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.2, wid1=0.1, weights=1/data['Int_err'])
ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

In [ ]:
# Cut through A
Q1 = np.array([0.5,0.5,0.5])  #Q-position

width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin = 5.5 #Minimum of energy
EMax = 7.5 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dset.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)

# Fit to a Gaussian and plot
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.8, wid1=0.1, weights=1/data['Int_err'])
ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

## Import and fit to high-resolution (HHL) data

In [ ]:
# High-resolution
files = _tools.fileListGenerator("2644-2659",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHR = DataSet.DataSet(files)
dsHR.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
dsHR.mask = [np.logical_or(m1,m2) for m1,m2 in zip(dsHR.mask,np.sum(mask)(dsHR))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHR,treshold)
print(i)

In [ ]:
# Plot QE slice
%matplotlib qt
dset = dsHR
rlu = True
#grid = -10
grid = False
self = dset
dataFiles = None
rlu = True
Q1 = np.array([0.5,0.5,1]) #Q1 of cut
Q2 = np.array([0.5,0.5,0.5]) #Q2 of cut
EMin = 5.0 #Energy minimum
EMax = 7.3 #Energy maximum
dE = 0.04 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.05 #step along Q in 1/A
vmin = 0 #caxis minimum
vmax = 1 #caxis maximum

ax,data,bins = dset.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=True,cmap='turbo')
ax.set_ylabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=16)
ax.set_xlabel("")
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_clim(vmin, vmax)
ax.set_xlim(0, 1)
ax.get_figure().set_size_inches(10,5)
ax.set_xticks([0, 1], ("M", "A"))
cb = ax.colorbar
cb.set_label(label='$I$ (arb. units)', size=16)
cb.ax.tick_params(labelsize=16)
cb.set_ticks([vmin, vmax])
plt.show()

In [ ]:
# Cut through M
dset = dsHR
Q1 = np.array([0.5,0.5,1])  #Q-position
width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin = 5.6 #Minimum of energy
EMax = 6.8 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dset.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
# Fit to a Gaussian and plot
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.2, wid1=0.1, weights=1/data['Int_err'])
ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

# Intermediate cuts
Q1 = np.array([0.5,0.5,0.9])  #Q-position
width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin = 5.6 #Minimum of energy
EMax = 7.0 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dset.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
# Fit to a Gaussian and plot
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.2, wid1=0.1, weights=1/data['Int_err'])
ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

Q1 = np.array([0.5,0.5,0.8])  #Q-position
width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin = 5.8 #Minimum of energy
EMax = 7.5 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dset.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
# Fit to a Gaussian and plot
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.4, wid1=0.1, weights=1/data['Int_err'])
ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

Q1 = np.array([0.5,0.5,0.7])  #Q-position
width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin = 6.0 #Minimum of energy
EMax = 7.5 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dset.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
# Fit to a Gaussian and plot
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.7, wid1=0.1, weights=1/data['Int_err'])
ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

Q1 = np.array([0.5,0.5,0.6])  #Q-position
width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin = 6.4 #Minimum of energy
EMax = 7.5 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dset.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
# Fit to a Gaussian and plot
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.9, wid1=0.1, weights=1/data['Int_err'])
ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")

# Cut through A
Q1 = np.array([0.5,0.5,0.5])  #Q-position
width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin = 6.4 #Minimum of energy
EMax = 7.5 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dset.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
# Fit to a Gaussian and plot
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.8, wid1=0.1, weights=1/data['Int_err'])
ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")